In [50]:
import pandas as pd
import html
pd.set_option('display.max_columns', None)

player_info_1617 = pd.read_excel('../../data/raw/player_name/players_1617.xlsx')
player_info_1718 = pd.read_excel('../../data/raw/player_name/players_1718.xlsx')
player_info_1819 = pd.read_excel('../../data/raw/player_name/players_1819.xlsx')
player_info_1920 = pd.read_excel('../../data/raw/player_name/players_1920.xlsx')

player_info_1617['name'] = player_info_1617['name'].apply(html.unescape)
player_info_1718['name'] = player_info_1718['name'].apply(html.unescape)
player_info_1819['name'] = player_info_1819['name'].apply(html.unescape)
player_info_1920['name'] = player_info_1920['name'].apply(html.unescape)

### Mapping player's position

In [51]:
pos_map = {
    'Goalkeeper':'GK',
    'Centre-Back':'DEF',
    'Left-Back':'DEF',
    'Right-Back':'DEF',
    
    'Defensive Midfield':'MID',
    'Central Midfield':'MID',
    'Attacking Midfield':'MID',
    'Right Midfield':'MID',
    'Left Midfield':'MID',
    
    'Left Winger':'FWD',
    'Right Winger':'FWD',
    'Centre-Forward':'FWD',
    'Second Striker':'FWD',
}

player_info_1617['position'] = player_info_1617['position'].map(pos_map)
player_info_1718['position'] = player_info_1718['position'].map(pos_map)
player_info_1819['position'] = player_info_1819['position'].map(pos_map)
player_info_1920['position'] = player_info_1920['position'].map(pos_map)

In [52]:
df_1617 = pd.read_csv('../../data/processed/gameweek/combined_gws_1617.csv')
df_1718 = pd.read_csv('../../data/processed/gameweek/combined_gws_1718.csv')
df_1819 = pd.read_csv('../../data/processed/gameweek/combined_gws_1819.csv')
df_1920 = pd.read_csv('../../data/processed/gameweek/combined_gws_1920.csv')
df_2021 = pd.read_csv('../../data/processed/gameweek/combined_gws_2021.csv')
df_2122 = pd.read_csv('../../data/processed/gameweek/combined_gws_2122.csv')
df_2223 = pd.read_csv('../../data/processed/gameweek/combined_gws_2223.csv')
df_2324 = pd.read_csv('../../data/processed/gameweek/combined_gws_2324.csv')

### Mapping Player's Position & Team (2016/17 - 2019/20) 

In [53]:
from rapidfuzz import process

# Dictionary untuk mapping name dengan posisi dari player_info_df
team_to_position_1617 = dict(zip(player_info_1617['name'], player_info_1617['team']))
team_to_position_1718 = dict(zip(player_info_1718['name'], player_info_1718['team']))
team_to_position_1819 = dict(zip(player_info_1819['name'], player_info_1819['team']))
team_to_position_1920 = dict(zip(player_info_1920['name'], player_info_1920['team']))

# Fungsi untuk mencocokkan name dengan fuzzy matching
def get_name(name, team_to_position):
    match = process.extractOne(name, team_to_position.keys(), score_cutoff=50)
    if match:
        return team_to_position[match[0]]
    return None

# Terapkan fungsi untuk menambahkan kolom 'position' ke gws_df
df_1617['team'] = df_1617['name'].apply(lambda name: get_name(name, team_to_position_1617))
df_1718['team'] = df_1718['name'].apply(lambda name: get_name(name, team_to_position_1718))
df_1819['team'] = df_1819['name'].apply(lambda name: get_name(name, team_to_position_1819))
df_1920['team'] = df_1920['name'].apply(lambda name: get_name(name, team_to_position_1920))

In [54]:
# Dictionary untuk mapping nama dengan posisi dari player_info_df
name_to_position_1617 = dict(zip(player_info_1617['name'], player_info_1617['position']))
name_to_position_1718 = dict(zip(player_info_1718['name'], player_info_1718['position']))
name_to_position_1819 = dict(zip(player_info_1819['name'], player_info_1819['position']))
name_to_position_1920 = dict(zip(player_info_1920['name'], player_info_1920['position']))

# Fungsi untuk mencocokkan nama dengan fuzzy matching
def get_name(name, name_to_position):
    match = process.extractOne(name, name_to_position.keys(), score_cutoff=50)
    if match:
        return name_to_position[match[0]]
    return None

# Terapkan fungsi untuk menambahkan kolom 'position' ke gws_df
df_1617['position'] = df_1617['name'].apply(lambda name: get_name(name, name_to_position_1617))
df_1718['position'] = df_1718['name'].apply(lambda name: get_name(name, name_to_position_1718))
df_1819['position'] = df_1819['name'].apply(lambda name: get_name(name, name_to_position_1819))
df_1920['position'] = df_1920['name'].apply(lambda name: get_name(name, name_to_position_1920))

### Adjust Features

In [55]:
new_features_list = ['xP', 'expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded']
df_list = [df_1617, df_1718, df_1819, df_1920]
season_list = ['1617', '1718', '1819', '1920']

for df, season in zip(df_list, season_list):
    for feature in new_features_list:
        df[feature] = 0

In [56]:
df_list = [df_2021, df_2122, df_2223, df_2324]
season_list = ['20-21', '21-22', '22-23', '23-24']

for df, season in zip(df_list, season_list):
    df['season'] = f'20{season}'

In [57]:
df_list = [df_2021, df_2122]
new_features_list = ['expected_assists', 'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded']

for df in df_list:
    for feature in new_features_list:
        df[feature] = 0

In [58]:
cols = ['season', 'name', 'position', 'team', 'assists', 'bonus', 'bps',
        'clean_sheets', 'creativity', 'element', 'fixture', 'goals_conceded',
        'goals_scored', 'ict_index', 'influence', 'xP', 'expected_assists', 
        'expected_goal_involvements', 'expected_goals', 'expected_goals_conceded', 
        'kickoff_time', 'minutes', 'opponent_team', 'own_goals', 'penalties_missed', 
        'penalties_saved', 'red_cards', 'saves', 'selected', 'team_a_score',
        'team_h_score', 'threat', 'total_points', 'transfers_balance', 'transfers_in', 
        'transfers_out', 'value', 'was_home', 'yellow_cards', 'GW']

df_1617 = df_1617[cols]
df_1718 = df_1718[cols]
df_1819 = df_1819[cols]
df_1920 = df_1920[cols]
df_2021 = df_2021[cols]
df_2122 = df_2122[cols]
df_2223 = df_2223[cols]
df_2324 = df_2324[cols]

### Merge dataset (2016/17 - 2023/24)

In [59]:
df = pd.concat([df_1617, df_1718, df_1819, df_1920, df_2021, df_2122, df_2223, df_2324]).reset_index(drop=True)
df

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham United,0,0,0,0,0.0,454,10,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2016-08-15T19:00:00Z,0,4,0,0,0,0,0,14023,1.0,2.0,0.0,0,0,0,0,55,False,0,1
1,2016-17,Aaron Lennon,FWD,Everton FC,0,0,6,0,0.3,142,3,0,0,0.9,8.2,0.0,0.00,0.00,0.00,0.00,2016-08-13T14:00:00Z,15,17,0,0,0,0,0,13918,1.0,1.0,0.0,1,0,0,0,60,True,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal FC,0,0,5,0,4.9,16,8,3,0,3.0,2.2,0.0,0.00,0.00,0.00,0.00,2016-08-14T15:00:00Z,60,9,0,0,0,0,0,163170,4.0,3.0,23.0,2,0,0,0,80,True,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford FC,0,0,0,0,0.0,482,7,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2016-08-13T14:00:00Z,0,13,0,0,0,0,0,1051,1.0,1.0,0.0,0,0,0,0,50,False,0,1
4,2016-17,Abdul Rahman Baba,DEF,Chelsea FC,0,0,0,0,0.0,80,10,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2016-08-15T19:00:00Z,0,20,0,0,0,0,0,1243,1.0,2.0,0.0,0,0,0,0,55,True,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196533,2023-24,Kyle Walker,DEF,Man City,0,0,11,0,3.1,369,86,1,0,1.2,8.6,2.3,0.01,0.01,0.00,0.83,2023-10-21T14:00:00Z,90,5,0,0,0,0,0,1669338,1.0,2.0,0.0,2,20093,85548,65455,53,True,0,9
196534,2023-24,Jacob Brown,FWD,Luton,0,0,1,0,2.3,631,88,1,0,3.2,2.2,1.3,0.01,0.39,0.38,2.87,2023-10-21T14:00:00Z,71,16,0,0,0,0,0,15587,2.0,2.0,27.0,2,-589,816,1405,49,False,0,9
196535,2023-24,Vicente Guaita,GK,Crystal Palace,0,0,0,0,0.0,227,87,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2023-10-21T14:00:00Z,0,15,0,0,0,0,0,30016,0.0,4.0,0.0,0,-2457,0,2457,44,False,0,9
196536,2023-24,Braian Ojeda Rodríguez,MID,Nott'm Forest,0,0,0,0,0.0,459,88,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2023-10-21T14:00:00Z,0,12,0,0,0,0,0,243,2.0,2.0,0.0,0,-8,0,8,45,True,0,9


In [60]:
df['position'] = df['position'].replace('GKP', 'GK')

### Mapping 'opponent_team' columns

In [61]:
team_id = pd.read_csv('../../data/raw/master_team_list.csv')

id_team_1617 = team_id[team_id['season'] == '2016-17']
id_team_1718 = team_id[team_id['season'] == '2017-18']
id_team_1819 = team_id[team_id['season'] == '2018-19']
id_team_1920 = team_id[team_id['season'] == '2019-20']
id_team_2021 = team_id[team_id['season'] == '2020-21']
id_team_2122 = team_id[team_id['season'] == '2021-22']
id_team_2223 = team_id[team_id['season'] == '2022-23']
id_team_2324 = pd.read_csv('../../data/raw/2023-24/teams.csv')

In [62]:
team_id_1617_map = dict(zip(id_team_1617['team'], id_team_1617['team_name']))
team_id_1718_map = dict(zip(id_team_1718['team'], id_team_1718['team_name']))
team_id_1819_map = dict(zip(id_team_1819['team'], id_team_1819['team_name']))
team_id_1920_map = dict(zip(id_team_1920['team'], id_team_1920['team_name']))
team_id_2021_map = dict(zip(id_team_2021['team'], id_team_2021['team_name']))
team_id_2122_map = dict(zip(id_team_2122['team'], id_team_2122['team_name']))
team_id_2223_map = dict(zip(id_team_2223['team'], id_team_2223['team_name']))
team_id_2324_map = dict(zip(id_team_2324['id'], id_team_2324['name']))

df.loc[df['season'] == '2016-17', 'opponent_team'] = df.loc[df['season'] == '2016-17', 'opponent_team'].replace(team_id_1617_map)
df.loc[df['season'] == '2017-18', 'opponent_team'] = df.loc[df['season'] == '2017-18', 'opponent_team'].replace(team_id_1718_map)
df.loc[df['season'] == '2018-19', 'opponent_team'] = df.loc[df['season'] == '2018-19', 'opponent_team'].replace(team_id_1819_map)
df.loc[df['season'] == '2019-20', 'opponent_team'] = df.loc[df['season'] == '2019-20', 'opponent_team'].replace(team_id_1920_map)
df.loc[df['season'] == '2020-21', 'opponent_team'] = df.loc[df['season'] == '2020-21', 'opponent_team'].replace(team_id_2021_map)
df.loc[df['season'] == '2021-22', 'opponent_team'] = df.loc[df['season'] == '2021-22', 'opponent_team'].replace(team_id_2122_map)
df.loc[df['season'] == '2022-23', 'opponent_team'] = df.loc[df['season'] == '2022-23', 'opponent_team'].replace(team_id_2223_map)
df.loc[df['season'] == '2023-24', 'opponent_team'] = df.loc[df['season'] == '2023-24', 'opponent_team'].replace(team_id_2324_map)

C:\Users\marse\AppData\Local\Temp\ipykernel_27152\1494846578.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['Chelsea' 'Spurs' 'Liverpool' ... 'Chelsea' 'Sunderland' 'Arsenal']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[df['season'] == '2016-17', 'opponent_team'] = df.loc[df['season'] == '2016-17', 'opponent_team'].replace(team_id_1617_map)


### Editting 'team' and 'opponent_team' value

In [63]:
replace_team_value = {
    'AFC Bournemouth':'Bournemouth', 
    'Arsenal FC':'Arsenal', 
    'Brighton':'Brighton & Hove Albion', 
    'Burnley FC':'Burnley', 
    'Chelsea FC':'Chelsea',
    'Everton FC':'Everton', 
    'Fulham FC':'Fulham',
    'Leeds':'Leeds United', 
    'Leicester':'Leicester City',
    'Liverpool FC':'Liverpool', 
    'Luton':'Luton Town', 
    'Man City':'Manchester City',
    'Man Utd':'Manchester United', 
    'Middlesbrough FC':'Middlesbrough', 
    'Newcastle':'Newcastle United', 
    'Norwich':'Norwich City', 
    "Nott'm Forest":'Nottingham Forest', 
    'Sheffield Utd':'Sheffield United',  
    'Southampton FC':'Southampton', 
    'Spurs':'Tottenham Hotspur',
    'Sunderland AFC':'Sunderland', 
    'Watford FC':'Watford', 
    'West Brom':'West Bromwich Albion',
    'West Ham':'West Ham United',
    'Wolves':'Wolverhampton Wanderers',
    'Huddersfield':'Huddersfield Town',
    'Hull':'Hull City',
    'Stoke':'Stoke City',
    'Swansea':'Swansea City',
    'Cardiff':'Cardiff City',
}

df['team'] = df['team'].replace(replace_team_value)
df['opponent_team'] = df['opponent_team'].replace(replace_team_value)

In [64]:
df['was_home'] = df['was_home'].map({False:0, True:1})

In [65]:
df

,season,name,position,team,assists,bonus,bps,clean_sheets,creativity,element,fixture,goals_conceded,goals_scored,ict_index,influence,xP,expected_assists,expected_goal_involvements,expected_goals,expected_goals_conceded,kickoff_time,minutes,opponent_team,own_goals,penalties_missed,penalties_saved,red_cards,saves,selected,team_a_score,team_h_score,threat,total_points,transfers_balance,transfers_in,transfers_out,value,was_home,yellow_cards,GW
0,2016-17,Aaron Cresswell,DEF,West Ham United,0,0,0,0,0.0,454,10,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2016-08-15T19:00:00Z,0,Chelsea,0,0,0,0,0,14023,1.0,2.0,0.0,0,0,0,0,55,0,0,1
1,2016-17,Aaron Lennon,FWD,Everton,0,0,6,0,0.3,142,3,0,0,0.9,8.2,0.0,0.00,0.00,0.00,0.00,2016-08-13T14:00:00Z,15,Tottenham Hotspur,0,0,0,0,0,13918,1.0,1.0,0.0,1,0,0,0,60,1,0,1
2,2016-17,Aaron Ramsey,MID,Arsenal,0,0,5,0,4.9,16,8,3,0,3.0,2.2,0.0,0.00,0.00,0.00,0.00,2016-08-14T15:00:00Z,60,Liverpool,0,0,0,0,0,163170,4.0,3.0,23.0,2,0,0,0,80,1,0,1
3,2016-17,Abdoulaye Doucouré,MID,Watford,0,0,0,0,0.0,482,7,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2016-08-13T14:00:00Z,0,Southampton,0,0,0,0,0,1051,1.0,1.0,0.0,0,0,0,0,50,0,0,1
4,2016-17,Abdul Rahman Baba,DEF,Chelsea,0,0,0,0,0.0,80,10,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2016-08-15T19:00:00Z,0,West Ham United,0,0,0,0,0,1243,1.0,2.0,0.0,0,0,0,0,55,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196533,2023-24,Kyle Walker,DEF,Manchester City,0,0,11,0,3.1,369,86,1,0,1.2,8.6,2.3,0.01,0.01,0.00,0.83,2023-10-21T14:00:00Z,90,Brighton & Hove Albion,0,0,0,0,0,1669338,1.0,2.0,0.0,2,20093,85548,65455,53,1,0,9
196534,2023-24,Jacob Brown,FWD,Luton Town,0,0,1,0,2.3,631,88,1,0,3.2,2.2,1.3,0.01,0.39,0.38,2.87,2023-10-21T14:00:00Z,71,Nottingham Forest,0,0,0,0,0,15587,2.0,2.0,27.0,2,-589,816,1405,49,0,0,9
196535,2023-24,Vicente Guaita,GK,Crystal Palace,0,0,0,0,0.0,227,87,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2023-10-21T14:00:00Z,0,Newcastle United,0,0,0,0,0,30016,0.0,4.0,0.0,0,-2457,0,2457,44,0,0,9
196536,2023-24,Braian Ojeda Rodríguez,MID,Nottingham Forest,0,0,0,0,0.0,459,88,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00,2023-10-21T14:00:00Z,0,Luton Town,0,0,0,0,0,243,2.0,2.0,0.0,0,-8,0,8,45,1,0,9


### Fixing Value (In Case There's a Wrong Assigned Value)

#### Fixing for ilogical position
> Ex: Player with Position 'DEF' have attribute 'saves' > 0

In [66]:
cek_df = df

In [67]:
cek_df['team'].sort_values().unique()

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Cardiff City', 'Chelsea',
       'Crystal Palace', 'Everton', 'Fulham', 'Huddersfield Town',
       'Hull City', 'Leeds United', 'Leicester City', 'Liverpool',
       'Luton Town', 'Manchester City', 'Manchester United',
       'Middlesbrough', 'Newcastle United', 'Norwich City',
       'Nottingham Forest', 'Sheffield United', 'Southampton',
       'Stoke City', 'Sunderland', 'Swansea City', 'Tottenham Hotspur',
       'Watford', 'West Bromwich Albion', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

In [68]:
cek_df[(cek_df['position'].isin(['FWD', 'MID', 'DEF'])) & (cek_df['saves'] > 0)]['name'].unique()

array(['Rui Pedro dos Santos Patrício'], dtype=object)

In [71]:
df.loc[(df['name'] == 'Rui Pedro dos Santos Patrício'), ['position', 'team']] = ['GK', 'Wolverhampton Wanderers']
df.loc[((df['name'] == 'Andy King') & (df['season'] == '2019-20')), ['position', 'team']] = ['MID', 'Leicester City']
df.loc[((df['name'] == 'Asmir Begovic') & (df['season'] == '2019-20')), ['position', 'team']] = ['GK', 'Bournemouth']
df.loc[((df['name'] == 'Joel Dinis Castro Pereira') & (df['season'] == '2017-18')), ['position', 'team']] = ['GK', 'West Ham United']
df.loc[((df['name'] == 'Darren Randolph') & (df['position'] == 'MID')), ['position', 'team']] = ['GK', 'Manchester United']
df.loc[((df['name'] == 'Frederico Rodrigues de Paula Santos')), ['position', 'team']] = ['MID', 'Manchester United']
df.loc[((df['name'] == 'João Filipe Iria Santos Moutinho')), ['position', 'team']] = ['MID', 'Wolverhampton Wanderers']

In [72]:
df.loc[((df['name'] == 'Manuel Agudo Durán') & (df['season'] == '2016-17')), ['position', 'team']] = ['FWD', 'Manchester City']
df.loc[((df['name'] == 'João Mário Naval Costa Eduardo') & (df['season'] == '2017-18')), ['position', 'team']] = ['MID', 'West Ham United']
df.loc[((df['name'] == 'Fabio Henrique Tavares') & (df['season'] == '2018-19')), ['position', 'team']] = ['MID', 'Liverpool']
df.loc[((df['name'] == 'Ricardo Domingos Barbosa Pereira') & (df['season'].isin(['2018-19', '2019-20']))), ['position', 'team']] = ['DEF', 'Leicester City']

#### Fixing value for each team

In [78]:
def teams_season(df, season):
    return df[df['season'] == season].sort_values(by='team')['team'].unique()

def check_team_players(df, season, team):
    return df[(df['team'] == team) & (df['season'] == season)].sort_values(by='name')['name'].unique()

##### 2016/17

In [79]:
teams_season(df, '2016-17')

array(['Arsenal', 'Bournemouth', 'Burnley', 'Chelsea', 'Crystal Palace',
       'Everton', 'Hull City', 'Leicester City', 'Liverpool',
       'Manchester City', 'Manchester United', 'Middlesbrough',
       'Southampton', 'Stoke City', 'Sunderland', 'Swansea City',
       'Tottenham Hotspur', 'Watford', 'West Bromwich Albion',
       'West Ham United'], dtype=object)

In [80]:
# check_team_players(df, '2016-17', 'Arsenal')
# check_team_players(df, '2016-17', 'Bournemouth')
# check_team_players(df, '2016-17', 'Burnley')
# check_team_players(df, '2016-17', 'Chelsea')
# check_team_players(df, '2016-17', 'Crystal Palace')
# check_team_players(df, '2016-17', 'Everton')
# check_team_players(df, '2016-17', 'Hull City')
# check_team_players(df, '2016-17', 'Leicester City')
# check_team_players(df, '2016-17', 'Liverpool')
# check_team_players(df, '2016-17', 'Manchester City')
# check_team_players(df, '2016-17', 'Manchester United')
# check_team_players(df, '2016-17', 'Middlesbrough')
# check_team_players(df, '2016-17', 'Southampton')
# check_team_players(df, '2016-17', 'Stoke City')
# check_team_players(df, '2016-17', 'Sunderland')
# check_team_players(df, '2016-17', 'Swansea City')
check_team_players(df, '2016-17', 'Tottenham Hotspur')
# check_team_players(df, '2016-17', 'Watford')
# check_team_players(df, '2016-17', 'West Bromwich Albion')
# check_team_players(df, '2016-17', 'West Ham United')

array(['Bamidele Alli', 'Ben Davies', 'Cameron Carter-Vickers',
       'Christian Eriksen', "Clinton N'Jie", 'Danny Rose', 'Eric Dier',
       'Erik Lamela', 'Filip Lesniak', 'Georges-Kévin Nkoudou',
       'Harry Kane', 'Harry Winks', 'Heung-Min Son', 'Hugo Lloris',
       'Jan Vertonghen', 'Joshua Onomah', 'Kevin Wimmer',
       'Kieran Trippier', 'Kyle Walker', 'Luke McGee', 'Michel Vorm',
       'Mousa Dembélé', 'Moussa Sissoko', 'Nabil Bentaleb',
       'Pau López Sabata', 'Samuel Shashoua', 'Toby Alderweireld',
       'Victor Wanyama', 'Vincent Janssen'], dtype=object)

##### 2017/18

In [81]:
teams_season(df, '2017-18')

array(['Arsenal', 'Bournemouth', 'Brighton & Hove Albion', 'Burnley',
       'Chelsea', 'Crystal Palace', 'Everton', 'Huddersfield Town',
       'Leicester City', 'Liverpool', 'Manchester City',
       'Manchester United', 'Newcastle United', 'Southampton',
       'Stoke City', 'Swansea City', 'Tottenham Hotspur', 'Watford',
       'West Bromwich Albion', 'West Ham United'], dtype=object)

In [82]:
check_team_players(df, '2017-18', 'Arsenal')
# check_team_players(df, '2017-18', 'Bournemouth')
# check_team_players(df, '2017-18', 'Brighton & Hove Albion')
# check_team_players(df, '2017-18', 'Burnley')
# check_team_players(df, '2017-18', 'Chelsea')
# check_team_players(df, '2017-18', 'Crystal Palace')
# check_team_players(df, '2017-18', 'Everton')
# check_team_players(df, '2017-18', 'Huddersfield Town')
# check_team_players(df, '2017-18', 'Leicester City')
# check_team_players(df, '2017-18', 'Liverpool')
# check_team_players(df, '2017-18', 'Manchester City')
# check_team_players(df, '2017-18', 'Manchester United')
# check_team_players(df, '2017-18', 'Newcastle United')
# check_team_players(df, '2017-18', 'Southampton')
# check_team_players(df, '2017-18', 'Stoke City')
# check_team_players(df, '2017-18', 'Swansea City')
# check_team_players(df, '2017-18', 'Tottenham Hotspur')
# check_team_players(df, '2017-18', 'Watford')
# check_team_players(df, '2017-18', 'West Bromwich Albion')
# check_team_players(df, '2017-18', 'West Ham United')

array(['Aaron Ramsey', 'Ainsley Maitland-Niles', 'Alex Iwobi',
       'Alex Oxlade-Chamberlain', 'Alexandre Lacazette', 'Alexis Sánchez',
       'Calum Chambers', 'Carl Jenkinson', 'Danny Welbeck',
       'David Ospina', 'Edward Nketiah', 'Francis Coquelin',
       'Granit Xhaka', 'Henrikh Mkhitaryan', 'Héctor Bellerín',
       'Jack Wilshere', 'Jordi Osei-Tutu', 'Joseph Willock',
       'Konstantinos Mavropanos', 'Laurent Koscielny', 'Marc Muniesa',
       'Mathieu Debuchy', 'Matt Macey', 'Mesut Özil', 'Mohamed Elneny',
       'Nacho Monreal', 'Olivier Giroud', 'Per Mertesacker', 'Petr Cech',
       'Pierre-Emerick Aubameyang', 'Reiss Nelson', 'Rob Holding',
       'Santiago Cazorla', 'Sead Kolasinac', 'Shkodran Mustafi'],
      dtype=object)

##### 2018/19

In [83]:
teams_season(df, '2018-19')

array(['Arsenal', 'Bournemouth', 'Brighton & Hove Albion', 'Burnley',
       'Cardiff City', 'Chelsea', 'Crystal Palace', 'Everton', 'Fulham',
       'Huddersfield Town', 'Leicester City', 'Liverpool',
       'Manchester City', 'Manchester United', 'Newcastle United',
       'Southampton', 'Tottenham Hotspur', 'Watford', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

In [84]:
check_team_players(df, '2018-19', 'Arsenal')
# check_team_players(df, '2018-19', 'Bournemouth')
# check_team_players(df, '2018-19', 'Brighton & Hove Albion')
# check_team_players(df, '2018-19', 'Burnley')
# check_team_players(df, '2018-19', 'Cardiff City')
# check_team_players(df, '2018-19', 'Chelsea')
# check_team_players(df, '2018-19', 'Crystal Palace')
# check_team_players(df, '2018-19', 'Everton')
# check_team_players(df, '2018-19', 'Fulham')
# check_team_players(df, '2018-19', 'Huddersfield Town')
# check_team_players(df, '2018-19', 'Leicester City')
# check_team_players(df, '2018-19', 'Liverpool')
# check_team_players(df, '2018-19', 'Manchester City')
# check_team_players(df, '2018-19', 'Manchester United')
# check_team_players(df, '2018-19', 'Newcastle United')
# check_team_players(df, '2018-19', 'Southampton')
# check_team_players(df, '2018-19', 'Tottenham Hotspur')
# check_team_players(df, '2018-19', 'Watford')
# check_team_players(df, '2018-19', 'West Ham United')
# check_team_players(df, '2018-19', 'Wolverhampton Wanderers')

array(['Aaron Ramsey', 'Ainsley Maitland-Niles', 'Alex Iwobi',
       'Alexandre Lacazette', 'Bernd Leno', 'Bukayo Saka',
       'Carl Jenkinson', 'Danny Welbeck', 'Denis Suárez',
       'Edward Nketiah', 'Emile Smith-Rowe', 'Emiliano Martínez',
       'Granit Xhaka', 'Henrikh Mkhitaryan', 'Héctor Bellerín',
       'Joseph Willock', 'Konstantinos Mavropanos', 'Laurent Koscielny',
       'Lucas Torreira', 'Matteo Guendouzi', 'Mesut Özil',
       'Mohamed Elneny', 'Nacho Monreal', 'Petr Cech',
       'Pierre-Emerick Aubameyang', 'Reiss Nelson', 'Rob Holding',
       'Sead Kolasinac', 'Shkodran Mustafi', 'Sokratis Papastathopoulos',
       'Stephan Lichtsteiner', 'Zech Medley'], dtype=object)

##### 2019/20

In [85]:
teams_season(df, '2019-20')

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brighton & Hove Albion',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton',
       'Leicester City', 'Liverpool', 'Manchester City',
       'Manchester United', 'Newcastle United', 'Norwich City',
       'Sheffield United', 'Southampton', 'Tottenham Hotspur', 'Watford',
       'West Ham United', 'Wolverhampton Wanderers'], dtype=object)

In [86]:
check_team_players(df, '2019-20', 'Arsenal')
# check_team_players(df, '2019-20', 'Aston Villa')
# check_team_players(df, '2019-20', 'Bournemouth')
# check_team_players(df, '2019-20', 'Brighton & Hove Albion')
# check_team_players(df, '2019-20', 'Burnley')
# check_team_players(df, '2019-20', 'Chelsea')
# check_team_players(df, '2019-20', 'Crystal Palace')
# check_team_players(df, '2019-20', 'Everton')
# check_team_players(df, '2019-20', 'Leicester City')
# check_team_players(df, '2019-20', 'Liverpool')
# check_team_players(df, '2019-20', 'Manchester City')
# check_team_players(df, '2019-20', 'Manchester United')
# check_team_players(df, '2019-20', 'Newcastle United')
# check_team_players(df, '2019-20', 'Norwich City')
# check_team_players(df, '2019-20', 'Sheffield United')
# check_team_players(df, '2019-20', 'Southampton')
# check_team_players(df, '2019-20', 'Tottenham Hotspur')
# check_team_players(df, '2019-20', 'Watford')
# check_team_players(df, '2019-20', 'West Ham United')
# check_team_players(df, '2019-20', 'Wolverhampton Wanderers')

array(['Ainsley Maitland-Niles', 'Alexandre Lacazette', 'Bernd Leno',
       'Bukayo Saka', 'Calum Chambers', 'Daniel Ceballos Fernández',
       'David Luiz Moreira Marinho', 'Edward Nketiah', 'Emile Smith Rowe',
       'Emiliano Martínez', 'Fernando Luiz Rosa', 'Granit Xhaka',
       'Henrikh Mkhitaryan', 'Héctor Bellerín', 'Jorge Luiz Frello Filho',
       'Joseph Willock', 'Kieran Tierney', 'Kieran Trippier',
       'Konstantinos Mavropanos', 'Lucas Torreira', 'Matt Macey',
       'Matteo Guendouzi', 'Matthew Smith', 'Mesut Özil', 'Nacho Monreal',
       'Nicolas Pépé', 'Pablo Marí', 'Pierre-Emerick Aubameyang',
       'Reiss Nelson', 'Rob Holding', 'Sead Kolasinac',
       'Shkodran Mustafi', 'Sokratis Papastathopoulos',
       'Tyreece John-Jules', 'Zech Medley'], dtype=object)

##### 2020/21

In [87]:
teams_season(df, '2020-21')

array(['Arsenal', 'Aston Villa', 'Brighton & Hove Albion', 'Burnley',
       'Chelsea', 'Crystal Palace', 'Everton', 'Fulham', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City',
       'Manchester United', 'Newcastle United', 'Sheffield United',
       'Southampton', 'Tottenham Hotspur', 'West Bromwich Albion',
       'West Ham United', 'Wolverhampton Wanderers'], dtype=object)

In [88]:
check_team_players(df, '2020-21', 'Arsenal')
# check_team_players(df, '2020-21', 'Aston Villa')
# check_team_players(df, '2020-21', 'Brighton & Hove Albion')
# check_team_players(df, '2020-21', 'Burnley')
# check_team_players(df, '2020-21', 'Chelsea')
# check_team_players(df, '2020-21', 'Crystal Palace')
# check_team_players(df, '2020-21', 'Everton')
# check_team_players(df, '2020-21', 'Fulham')
# check_team_players(df, '2020-21', 'Leeds United')
# check_team_players(df, '2020-21', 'Leicester City')
# check_team_players(df, '2020-21', 'Liverpool')
# check_team_players(df, '2020-21', 'Manchester City')
# check_team_players(df, '2020-21', 'Manchester United')
# check_team_players(df, '2020-21', 'Newcastle United')
# check_team_players(df, '2020-21', 'Sheffield United')
# check_team_players(df, '2020-21', 'Southampton')
# check_team_players(df, '2020-21', 'Tottenham Hotspur')
# check_team_players(df, '2020-21', 'West Bromwich Albion')
# check_team_players(df, '2020-21', 'West Ham United')
# check_team_players(df, '2020-21', 'Wolverhampton Wanderers')

array(['Ainsley Maitland-Niles', 'Alexandre Lacazette', 'Bernd Leno',
       'Bukayo Saka', 'Calum Chambers', 'Cédric Soares',
       'Daniel Ceballos Fernández', 'David Luiz Moreira Marinho',
       'Edward Nketiah', 'Emile Smith Rowe', 'Emiliano Martínez',
       'Gabriel Magalhães', 'Gabriel Teodoro Martinelli Silva',
       'Granit Xhaka', 'Héctor Bellerín', 'Joseph Willock',
       'Kieran Tierney', 'Lucas Torreira', 'Martin Ødegaard',
       'Mathew Ryan', 'Matt Macey', 'Matteo Guendouzi', 'Mesut Özil',
       'Miguel Azeez', 'Mohamed Naser El Sayed Elneny', 'Nicolas Pépé',
       'Pablo Marí', 'Pierre-Emerick Aubameyang', 'Reiss Nelson',
       'Rob Holding', 'Rúnar Alex Rúnarsson', 'Sead Kolasinac',
       'Shkodran Mustafi', 'Sokratis Papastathopoulos', 'Thomas Partey',
       'William Saliba', 'Willian Borges Da Silva'], dtype=object)

##### 2021/22

In [89]:
teams_season(df, '2021-22')

array(['Arsenal', 'Aston Villa', 'Brentford', 'Brighton & Hove Albion',
       'Burnley', 'Chelsea', 'Crystal Palace', 'Everton', 'Leeds United',
       'Leicester City', 'Liverpool', 'Manchester City',
       'Manchester United', 'Newcastle United', 'Norwich City',
       'Southampton', 'Tottenham Hotspur', 'Watford', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

In [90]:
check_team_players(df, '2021-22', 'Arsenal')
# check_team_players(df, '2021-22', 'Aston Villa')
# check_team_players(df, '2021-22', 'Brentford')
# check_team_players(df, '2021-22', 'Brighton & Hove Albion')
# check_team_players(df, '2021-22', 'Burnley')
# check_team_players(df, '2021-22', 'Chelsea')
# check_team_players(df, '2021-22', 'Crystal Palace')
# check_team_players(df, '2021-22', 'Everton')
# check_team_players(df, '2021-22', 'Leeds United')
# check_team_players(df, '2021-22', 'Leicester City')
# check_team_players(df, '2021-22', 'Liverpool')
# check_team_players(df, '2021-22', 'Manchester City')
# check_team_players(df, '2021-22', 'Manchester United')
# check_team_players(df, '2021-22', 'Newcastle United')
# check_team_players(df, '2021-22', 'Norwich City')
# check_team_players(df, '2021-22', 'Southampton')
# check_team_players(df, '2021-22', 'Tottenham Hotspur')
# check_team_players(df, '2021-22', 'Watford')
# check_team_players(df, '2021-22', 'West Ham United')
# check_team_players(df, '2021-22', 'Wolverhampton Wanderers')

array(['Aaron Ramsdale', 'Ainsley Maitland-Niles', 'Albert Sambi Lokonga',
       'Alex Kirk', 'Alexandre Lacazette', 'Arthur Okonkwo', 'Ben White',
       'Bernd Leno', 'Bukayo Saka', 'Calum Chambers', 'Charlie Patino',
       'Cédric Soares', 'Edward Nketiah', 'Emile Smith Rowe',
       'Folarin Balogun', 'Gabriel Magalhães',
       'Gabriel Teodoro Martinelli Silva', 'Granit Xhaka',
       'Héctor Bellerín', 'Karl Hein', 'Kieran Tierney',
       'Konstantinos Mavropanos', 'Lucas Torreira', 'Marcelo Flores',
       'Martin Ødegaard', 'Mattéo Guendouzi', 'Mazeed Ogungbo',
       'Miguel Azeez', 'Mika Biereth', 'Mohamed Naser El Sayed Elneny',
       'Nicolas Pépé', 'Nuno Varela Tavares', 'Omari Hutchinson',
       'Pablo Marí', 'Pierre-Emerick Aubameyang', 'Reiss Nelson',
       'Rob Holding', 'Ryan Alebiosu', 'Rúnar Alex Rúnarsson',
       "Salah-Eddine Oulad M'hand", 'Sead Kolasinac', 'Takehiro Tomiyasu',
       'Thomas Partey', 'William Saliba', 'Willian Borges Da Silva',
       'Z

##### 2022/23

In [91]:
teams_season(df, '2022-23')

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Chelsea', 'Crystal Palace', 'Everton',
       'Fulham', 'Leeds United', 'Leicester City', 'Liverpool',
       'Manchester City', 'Manchester United', 'Newcastle United',
       'Nottingham Forest', 'Southampton', 'Tottenham Hotspur',
       'West Ham United', 'Wolverhampton Wanderers'], dtype=object)

In [92]:
check_team_players(df, '2022-23', 'Arsenal')
# check_team_players(df, '2022-23', 'Aston Villa')
# check_team_players(df, '2022-23', 'Bournemouth')
# check_team_players(df, '2022-23', 'Brentford')
# check_team_players(df, '2022-23', 'Brighton & Hove Albion')
# check_team_players(df, '2022-23', 'Chelsea')
# check_team_players(df, '2022-23', 'Crystal Palace')
# check_team_players(df, '2022-23', 'Everton')
# check_team_players(df, '2022-23', 'Fulham')
# check_team_players(df, '2022-23', 'Leeds United')
# check_team_players(df, '2022-23', 'Leicester City')
# check_team_players(df, '2022-23', 'Liverpool')
# check_team_players(df, '2022-23', 'Manchester City')
# check_team_players(df, '2022-23', 'Manchester United')
# check_team_players(df, '2022-23', 'Newcastle United')
# check_team_players(df, '2022-23', 'Nottingham Forest')
# check_team_players(df, '2022-23', 'Southampton')
# check_team_players(df, '2022-23', 'Tottenham Hotspur')
# check_team_players(df, '2022-23', 'West Ham United')
# check_team_players(df, '2022-23', 'Wolverhampton Wanderers')

array(['Aaron Ramsdale', 'Ainsley Maitland-Niles', 'Albert Sambi Lokonga',
       'Amario Cozier-Duberry', 'Benjamin White', 'Bukayo Saka',
       'Cédric Alves Soares', 'Eddie Nketiah', 'Emile Smith Rowe',
       'Ethan Nwaneri', 'Fábio Ferreira Vieira',
       'Gabriel Fernando de Jesus', 'Gabriel Martinelli Silva',
       'Gabriel dos Santos Magalhães', 'Granit Xhaka', 'Jakub Kiwior',
       'Jorge Luiz Frello Filho', 'Karl Hein', 'Kieran Tierney',
       'Leandro Trossard', 'Lino Sousa', 'Lucas Torreira di Pascua',
       'Marcus Oliveira Alencar', 'Martin Ødegaard', 'Matt Turner',
       'Matthew Smith', 'Mauro Bandeira', 'Mohamed Elneny',
       'Nathan Butler-Oyedeji', 'Nicolas Pépé', 'Nuno Varela Tavares',
       'Oleksandr Zinchenko', 'Pablo Marí Villar', 'Reiss Nelson',
       'Reuell Walters', 'Rob Holding', 'Takehiro Tomiyasu',
       'Thomas Partey', 'William Saliba'], dtype=object)

##### 2023/24

In [93]:
teams_season(df, '2023-24')

array(['Arsenal', 'Aston Villa', 'Bournemouth', 'Brentford',
       'Brighton & Hove Albion', 'Burnley', 'Chelsea', 'Crystal Palace',
       'Everton', 'Fulham', 'Liverpool', 'Luton Town', 'Manchester City',
       'Manchester United', 'Newcastle United', 'Nottingham Forest',
       'Sheffield United', 'Tottenham Hotspur', 'West Ham United',
       'Wolverhampton Wanderers'], dtype=object)

In [94]:
check_team_players(df, '2023-24', 'Arsenal')
# check_team_players(df, '2023-24', 'Aston Villa')
# check_team_players(df, '2023-24', 'Bournemouth')
# check_team_players(df, '2023-24', 'Brentford')
# check_team_players(df, '2023-24', 'Brighton & Hove Albion')
# check_team_players(df, '2023-24', 'Burnley')
# check_team_players(df, '2023-24', 'Chelsea')
# check_team_players(df, '2023-24', 'Crystal Palace')
# check_team_players(df, '2023-24', 'Everton')
# check_team_players(df, '2023-24', 'Fulham')
# check_team_players(df, '2023-24', 'Liverpool')
# check_team_players(df, '2023-24', 'Luton Town')
# check_team_players(df, '2023-24', 'Manchester City')
# check_team_players(df, '2023-24', 'Manchester United')
# check_team_players(df, '2023-24', 'Newcastle United')
# check_team_players(df, '2023-24', 'Nottingham Forest')
# check_team_players(df, '2023-24', 'Sheffield United')
# check_team_players(df, '2023-24', 'Tottenham Hotspur')
# check_team_players(df, '2023-24', 'West Ham United')
# check_team_players(df, '2023-24', 'Wolverhampton Wanderers')

array(['Aaron Ramsdale', 'Albert Sambi Lokonga', 'Benjamin White',
       'Bradley Ibrahim', 'Bukayo Saka', 'Charles Sagoe',
       'Cédric Alves Soares', 'David Raya Martin', 'Declan Rice',
       'Eddie Nketiah', 'Emile Smith Rowe', 'Ethan Nwaneri',
       'Folarin Balogun', 'Fábio Ferreira Vieira',
       'Gabriel Fernando de Jesus', 'Gabriel Martinelli Silva',
       'Gabriel dos Santos Magalhães', 'Granit Xhaka', 'Jakub Kiwior',
       'James Sweet', 'Jorge Luiz Frello Filho', 'Jurriën Timber',
       'Kai Havertz', 'Karl Hein', 'Kieran Tierney', 'Leandro Trossard',
       'Marcus Oliveira Alencar', 'Martin Ødegaard', 'Mauro Bandeira',
       'Mohamed Elneny', 'Myles Lewis-Skelly', 'Nicolas Pépé',
       'Nuno Varela Tavares', 'Oleksandr Zinchenko', 'Reiss Nelson',
       'Reuell Walters', 'Rob Holding', 'Rúnar Alex Rúnarsson',
       'Takehiro Tomiyasu', 'Thomas Partey', 'William Saliba'],
      dtype=object)

### Save Cleaned Dataset

In [104]:
df.to_csv('../../data/processed/final-dataset.csv', index=False)